## 1. Python Networking Using TCP

#### What is TCP?
TCP stands for Transmission Control Protocol. It is a connection-oriented protocol that provides reliable, ordered, and error-checked delivery of a stream of data between applications running on hosts communicating via an IP network.

## Understanding Sockets, AF_INET, SOCK_STREAM, and SOCK_DGRAM

#### Sockets
- Sockets are endpoints for sending and receiving data across a network. They provide a way for applications to communicate over a network, using standard protocols like TCP and UDP. A socket is defined by an IP address and a port number, which together uniquely identify a network connection.

#### AF_INET
- AF_INET stands for "Address Family - Internet". It specifies the address family for IPv4. When creating a socket, `AF_INET` is used to indicate that the socket will use the IPv4 protocol for communication.

#### SOCK_STREAM
- SOCK_STREAM is used to specify a TCP (Transmission Control Protocol) socket type. TCP sockets are connection-oriented, providing reliable, ordered, and error-checked delivery of a stream of bytes between applications. When you use `SOCK_STREAM`, you are creating a socket that supports TCP.

### SOCK_DGRAM
- SOCK_DGRAM is used to specify a UDP (User Datagram Protocol) socket type. UDP sockets are connectionless, providing an unreliable, unordered, and error-prone way of sending datagrams (discrete packets of data). When you use `SOCK_DGRAM`, you are creating a socket that supports UDP.


### Creating a TCP Server

In [ ]:
import socket

def start_server(host='127.0.0.1', port=65432):
    
    #Dynamically get server ip address
    #host = socket.gethostbyname(socket.gethostname())

    # Create a socket object
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    # Bind the socket to the address and port
    server_socket.bind((host, port))
    
    # Listen for incoming connections (max 5 queued connections)
    server_socket.listen(5)
    print(f"Server started, listening on {host}:{port}")
    
    while True:
        # Accept a new connection
        client_socket, client_address = server_socket.accept()
        print(f"Connection from {client_address} has been established.")
        
        # Receive data from the client
        data = client_socket.recv(1024)
        print(f"Received data: {data.decode('utf-8')}")
        
        # Send a response to the client
        response = "Message received"
        client_socket.sendall(response.encode('utf-8'))
        
        # Close the connection
        client_socket.close()

# Run the server
if __name__ == "__main__":
    start_server()


### Explanation
- socket.socket(): Creates a new socket using the given address family (AF_INET for IPv4) and socket type (SOCK_STREAM for TCP).
- bind(): Binds the socket to an address and port.
- listen(): Enables the server to accept connections. The parameter specifies the maximum number of queued connections.
- accept(): Accepts an incoming connection and returns a new socket object and the address of the client.
- recv(): Receives data from the client.
- sendall(): Sends data to the client.
- close(): Closes the connection.

### Creating a TCP Client
A TCP client connects to a server, sends data, and receives a response.

In [ ]:
import socket

def start_client(host='127.0.0.1', port=65432):
    # Create a socket object
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    # Connect to the server
    client_socket.connect((host, port))
    print(f"Connected to {host}:{port}")
    
    # Send data to the server
    message = "Hello, Server!"
    client_socket.sendall(message.encode('utf-8'))
    
    # Receive response from the server
    response = client_socket.recv(1024)
    print(f"Received response: {response.decode('utf-8')}")
    
    # Close the connection
    client_socket.close()

# Run the client
if __name__ == "__main__":
    start_client()


### Explanation
- connect(): Connects to a remote socket at the given address and port.
- sendall(): Sends data to the server.
- recv(): Receives data from the server.

### Running the Server and Client

1. Start the Server: Run the server script first. It will start listening for incoming connections.
   
    ##### python tcp_server.py

3. Start the Client: Run the client script in another terminal or on another machine.
   
    ##### python tcp_client.py

When the client connects to the server and sends a message, the server will receive the message, print it, and send a response back to the client.

## 2. Python Networking Using TCP with Threading

### Why Use Threading?
Threading allows a program to run multiple operations concurrently in separate threads. In the context of a TCP server, threading enables the server to handle multiple client connections at the same time, improving efficiency and responsiveness.

### Creating a Multi-threaded TCP Server
A TCP server listens for incoming client connections on a specified IP address and port. Here’s how you can create a multi-threaded TCP server in Python:

In [ ]:
import socket
import threading

def handle_client(client_socket, client_address):
    print(f"Connection from {client_address} has been established.")
    
    while True:
        # Receive data from the client
        data = client_socket.recv(1024)
        if not data:
            break
        print(f"Received data from {client_address}: {data.decode('utf-8')}")
        
        # Send a response to the client
        response = "Message received"
        client_socket.sendall(response.encode('utf-8'))
    
    # Close the connection
    print(f"Connection from {client_address} has been closed.")
    client_socket.close()

def start_server(host='127.0.0.1', port=65432):
    # Create a socket object
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    # Bind the socket to the address and port
    server_socket.bind((host, port))
    
    # Listen for incoming connections (max 5 queued connections)
    server_socket.listen(5)
    print(f"Server started, listening on {host}:{port}")
    
    while True:
        # Accept a new connection
        client_socket, client_address = server_socket.accept()
        
        # Create a new thread to handle the client connection
        client_handler = threading.Thread(target=handle_client, args=(client_socket, client_address))
        client_handler.start()

# Run the server
if __name__ == "__main__":
    start_server()


### send() vs sendall()

1. **`send(data)`**:
   - This method sends the given data over the socket.
   - It returns the number of bytes sent.
   - If the socket's send buffer is full, `send()` may only send a part of the data, and the number of bytes sent may be less than the length of the data.
   - It is the responsibility of the programmer to handle cases where not all data is sent in a single call to `send()`.
   
2. **`sendall(data)`**:
   - This method sends the entire contents of the given data over the socket.
   - Unlike `send()`, `sendall()` will continue sending data until all of it has been sent or an error occurs.
   - It ensures that all the data is sent in one go, and it handles the case where the socket's send buffer may be full by repeatedly calling `send()` until all data is sent or an error occurs.
   - It returns `None`.
   - It is a convenience method that simplifies sending large chunks of data over a socket.
